# Extract information from Chinese catalogue card xmls

In [1]:
from __future__ import annotations
import sys
if '../' not in sys.path:
    sys.path.append('../')
import os
import glob
import re 
import io
import xml.etree.ElementTree as ET
from IPython.display import Image
from urllib.parse import quote
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from pymarc import marcxml, Record
from tqdm import tqdm
from dotenv import load_dotenv
import bookops_worldcat as bw

In [2]:
tqdm.pandas()

## Parsing xmls

In [ ]:
p5_local = "..\\data\\raw\\chinese"

In [ ]:
attempts = 0
# page_xml_loc = os.path.join(p5_root, "page")
page_xml_loc = os.path.join(p5_local, "1016992")
while attempts < 3:
    xmls = glob.glob(os.path.join(p5_local, "1016992", "*.pxml"))
    if len(xmls) > 0:
        break
    else:
        attempts += 1
        continue
else:
    raise IOError(f"Failed to connect to {page_xml_loc}")
    
xmls

In [ ]:
xmlroots = []
print(f"\nGetting xml roots from {page_xml_loc}")
for file in tqdm(xmls):
    fileName = os.fsdecode(file)
    attempts = 0
    while attempts < 3:
        try:
            tree = ET.parse(fileName)
            break
        except FileNotFoundError:
            attempts += 1
            continue
    else:
        raise FileNotFoundError(f"Failed to connect to: {fileName}")
    root = tree.getroot()
    xmlroots.append(root)

In [ ]:
len(xmlroots)

In [ ]:
def extractLines(root: xml.etree.ElementTree.Element):
    lines = []

    textRegions = [x for x in root[1] if len(x) > 2]  # Empty Text Regions Removed

    for textRegion in textRegions:
        textLines = textRegion[1:-1]  # Skip coordinate data in first child
        for textLine in textLines:
            lines.append(textLine[-1][0].text)  # Text equivalent for line
    return lines

def extractLinesForVol(vol: list[xml.etree.ElementTree.Element]):
    allLines = []
    for root in tqdm(vol):
        rootLines = extractLines(root)
        allLines.append(rootLines) 
    return allLines

In [ ]:
def extractLinesForVol(vol: list[xml.etree.ElementTree.Element]):
    allLines = []
    for root in tqdm(vol):
        rootLines = extractLines(root)
        allLines.append(rootLines) 
    return allLines

In [ ]:
caps_regex = re.compile("[A-Z][A-Z][A-Z]+")
c_num_regex = re.compile("C\.[0-9]")  # C number title references
i_num_regex = re.compile("I[ABC]\.\s[0-9]")  # I number title references
date_regex = re.compile("1[45][0-9][0-9]")  # Date format regexes (specific to this volume)
smark_regex = re.compile("[0-9]{1,5}[\s\.]{1,2}[\w]{1,3}[\s\.]{1,2}[\w0-9]{1,5}")
author_regex = re.compile("[A-Z]+[\s]+\([A-Z][a-z]+\)")
isbn_regex = re.compile("ISBN\s[0-9\-\s]+")

In [ ]:
cards = extractLinesForVol(xmlroots)

In [ ]:
cards_df_v0 = pd.DataFrame(
    data={
        "xml": [os.path.basename(x) for x in xmls],
        "lines": cards,
        "dummy": [None for x in cards]
    }
)

In [ ]:
def find_author(lines, dummy):
    author, title = None, None
    
    for i, l in enumerate(lines):
        if author_regex.search(l):  # look for an author format match
            author = l
            break
    
    if author:
        if i >= 2:  # author is after the second line (where we expect the title)
            title = " ".join(lines[1:i])
        elif i == 1:  # author is the second line
            title = lines[2]
    else:
        title = lines[1]  # default to the title being the second line
        
    return title, author

In [ ]:
def isbn_search(x):
    if type(x) is not list:
        raise TypeError(f"List expected not {type(x)}")
    res = isbn_regex.search("".join(x))
    if res:
        return res.group().replace("-", "").replace(" ", "").lstrip("ISBN")
    else:
        return None

In [ ]:
def shelfmark_search(x):
    if smark_regex.search(x[0]):
        result = smark_regex.search(x[0]).group()
    elif smark_regex.search(x[1]):
        result = smark_regex.search(x[1]).group()
    else:
        return None
    return result.replace(" ", "")

In [ ]:
cards_df_v0["shelfmark"] = cards_df_v0["lines"].transform(shelfmark_search)
t_a = cards_df_v0.loc[:,('lines', 'dummy')].transform(lambda x: find_author(x[0], x[1]), axis=1).rename(columns={"lines":"title", "dummy":"author"})
cards_df = cards_df_v0.drop(columns="dummy").join(t_a)
cards_df["isbn"] = cards_df["lines"].transform(lambda x:isbn_search(x))
cards_df["title"] = cards_df["title"].str.replace("\"", "\'")

In [ ]:
cards_df.info()

## OCLC API Queries

In [3]:
cards_df = pickle.load(open("401_cards.p", "rb"))

In [4]:
load_dotenv()
client_id = os.environ["CLIENT_ID"]
client_secret = os.environ["CLIENT_SECRET"]

In [5]:
token = bw.WorldcatAccessToken(
    key=client_id,
    secret=client_secret,
    scopes=["WorldCatMetadataAPI"],
    principal_id="unused",
    principal_idns="unused"
)

In [6]:
session = bw.MetadataSession(authorization=token)

In [7]:
def list_formats(recs):
    if not recs.get("briefRecords"):
        return None
    else:
        return [(x.get("generalFormat"), x.get("specificFormat")) for x in recs["briefRecords"]]

formats = cards_df["brief_bibs"].apply(lambda x:list_formats(x))
set(formats.dropna().sum())

{('ArtChap', 'Artcl'),
 ('Book', 'Digital'),
 ('Book', 'Mic'),
 ('Book', 'PrintBook'),
 ('Book', 'thsis'),
 ('CompFile', None),
 ('Jrnl', 'Digital'),
 ('Jrnl', None),
 ('Video', 'DVD')}

In [ ]:
# item_subtypes = "artchap-artcl, book-mic, book-thsis, book-printbook, jrnl-print"

In [ ]:
# itemSubType devised by looking at itemSubType for all returned records
# help.oclc.org/Librarian_Toolbox/Searching_WorldCat_Indexes/Bibliographic_records/Format_Document_Type_values_and_codes/WorldShare_and_WorldCat_Discovery
search_kwargs = {
    "inCatalogLanguage":None,
    "limit": 50,
    "orderBy": "bestMatch",
    "itemSubType": "artchap-artcl, book-mic, book-thsis, book-printbook, jrnl-print"
}

In [ ]:
def apply_search_brief_bib(ti=None, au=None, isbn=None, session=None, search_kwargs={}):
    """
    search_brief_bib applicable to df
    Known issue with specifying offset/limit
    So specify acceptable itemSubTypes and hope correct result is in first 50 records
    """
    
    res = None
    
    if isbn:
        query = f'bn:{isbn}'
        res = session.search_brief_bibs(q=query, **search_kwargs)
        
    if not res or res.json()["numberOfRecords"] == 0:
        query = f'ti:"{ti}" and au:"{au}"'
        res = session.search_brief_bibs(q=query, **search_kwargs)
    
    return res.json()

def apply_get_full_bib(brief_bibs, session):
    if brief_bibs["numberOfRecords"] == 0:
        return None
    else:
        recs = brief_bibs["briefRecords"]
        matched_xml = [session.get_full_bib(rec["oclcNumber"]).text for rec in recs]
        matched_records = [marcxml.parse_xml_to_array(io.StringIO(x))[0] for x in matched_xml]
        return matched_records

In [14]:
cards_df["brief_bibs"][0]

{'numberOfRecords': 16,
 'briefRecords': [{'oclcNumber': '222378954',
   'title': '風陵渡',
   'creator': '端木蕻良, 1912-',
   'date': 'Minguo 28 [1939]',
   'machineReadableDate': 'Minguo 28 1939',
   'language': 'chi',
   'generalFormat': 'Book',
   'specificFormat': 'PrintBook',
   'edition': 'Chu ban',
   'publisher': '上海雜誌公司',
   'publicationPlace': 'Shanghai',
   'mergedOclcNumbers': ['19748304', '48008952'],
   'catalogingInfo': {'catalogingAgency': 'AU@',
    'catalogingLanguage': 'eng',
    'levelOfCataloging': 'M',
    'transcribingAgency': 'AU@'}},
  {'oclcNumber': '23921305',
   'title': '風陵渡',
   'creator': '端木蕻良, 1912-',
   'date': '1979 printing',
   'machineReadableDate': '1979 printing',
   'language': 'chi',
   'generalFormat': 'Book',
   'specificFormat': 'PrintBook',
   'edition': '再版',
   'publisher': '文敎出版社 : 發行 天地圖書有限公司',
   'publicationPlace': 'Xianggang',
   'catalogingInfo': {'catalogingAgency': 'DLC',
    'catalogingLanguage': 'eng',
    'levelOfCataloging': ' ',
 

In [36]:
cards_df["worldcat_matches_subtyped"][0][0].fields[0].tag

'001'

In [12]:
print(cards_df["worldcat_matches_subtyped"][0][0])

=LDR  00000cam a2200000 i 4500
=001  ocm23921305
=003  OCoLC
=005  20220628065841.3
=008  910612s1979\\\\cc\\\\\\\\\\\\000\1\chi\\
=010  \\$a   80835064 
=040  \\$aDLC$beng$cCLU$dIOG$dOCLCQ$dOCLCG$dOCLCA$dOCLCQ$dOCLCO$dLOA$dOCLCF$dOCLCO$dOCLCQ$dOCLCO$dOCLCQ
=066  \\$c$1
=029  1\$aAU@$b000015296096
=035  \\$a(OCoLC)23921305
=050  04$aPL2815.A636$bF4 1979
=082  00$a359.d.65
=100  1\$6880-01$aDuanmu, Hongliang,$d1912-1996.
=245  10$6880-02$aFeng ling du /$c[Zhu zhe Duanmu Hongliang].
=260  \\$6880-03$aXianggang :$bWen jiao chu ban she :$bFa xing Tian di tu shu you xian gong si,$c1979 printing.
=300  \\$a1, 202, 4 pages ;$c19 cm
=336  \\$atext$btxt$2rdacontent
=337  \\$aunmediated$bn$2rdamedia
=338  \\$avolume$bnc$2rdacarrier
=490  1\$6880-04$aZhongguo xian dai wen xue cong shu (Wen jiao chu ban she)
=500  \\$aShort stories.
=505  0\$6880-05$aZui jiao -- Qing di -- Feng ling du -- Luo shi gu -- Huo tui -- Pao mo -- E xia -- Ke su xing de -- San yue ye qu.
=650  \0$aShort stories, Chinese$y

In [39]:
def count_field_occurences(lst):
    fields = []
    if lst:
        [fields.extend([y.tag for y in r.fields]) for r in lst]
    return fields

In [40]:
field_sum = cards_df["worldcat_matches_subtyped"].apply(count_field_occurences)

In [53]:
all_fields = pd.Series(index=field_sum.sum(), data=np.ones(len(field_sum.sum())))
#all_fields.groupby().count()

In [106]:
all_fields.groupby(level=0).count().head(50)#.sort_values(ascending=False).head(50)

001    1949
003    1949
005    1949
007      14
008    1949
010     198
015      18
016     170
019     150
020    1187
024       8
025       1
028       1
029     655
035    2006
037       4
040    1949
041     128
042      28
043     315
044       6
045       5
050     649
055      23
060       9
066    1678
070       1
072       6
080       6
082     656
084     730
090       5
092       3
100    1288
110      10
111       7
130       5
240      80
242      17
243       1
245    1949
246     961
250    1103
260    1652
264     207
300    1841
310       1
336    1114
337    1114
338     986
dtype: int64

In [86]:
def check_field_contents(lst):
    if lst:
        nat_bib_ctrl = []
        [nat_bib_ctrl.extend(r.get_fields("016")) for r in lst]
        if nat_bib_ctrl:
            return list(set(nat_bib_ctrl))
        else:
            return None
    else:
        return None

In [87]:
nat_bib_agencies = cards_df["worldcat_matches_subtyped"].apply(check_field_contents).dropna()

In [88]:
def find_2_subfield(lst):
    return [r.get_subfields("2") for r in lst]

In [97]:
subf_2 = pd.Series([x[0] for x in nat_bib_agencies.apply(find_2_subfield).sum() if x])

In [99]:
pd.Series(index=subf_2, data=np.ones(len(subf_2))).groupby(level=0).count()

ChTaNC      109
DNLM          1
FrPBN         7
JP-ToKJK     47
JTNDL         1
Uk            3
dtype: int64

In [ ]:
# cards_df["brief_bibs_subtyped"] = cards_df.progress_apply(lambda x: apply_search_brief_bib(x["title"], x["author"], x["isbn"], session=session, search_kwargs=search_kwargs), axis=1)
# cards_df["num_records_subtyped"] = cards_df["brief_bibs_subtyped"].apply(lambda x: x["numberOfRecords"])

In [ ]:
# cards_df["worldcat_matches_subtyped"] = cards_df["brief_bibs_subtyped"].progress_apply(apply_get_full_bib, session=session)

In [ ]:
# pickle.dump(cards_df, open("401_cards.p", "wb"))

### Explore bug with offset/limit
https://community.oclc.org/t5/oclc-apis-discussions/worldcat-metadata-search-brief-bibs-amp-limit-and-amp-offset/td-p/49236

In [ ]:
big_ones = cards_df[cards_df["num_records"] > 50].iloc[:2].progress_apply(lambda x: apply_search_brief_bib(x["title"], x["author"], x["isbn"], session=session), axis=1)

In [ ]:
big_ones

In [ ]:
query_test = f'ti:"FEI LONG QUAN ZHUAN" and au:"WU (Xuan)"'

In [ ]:
first_50 = session.search_brief_bibs(q=query_test, inCatalogLanguage=None, limit=50, orderBy="bestMatch")
first_50_oclc = [x["oclcNumber"] for x in first_50.json()["briefRecords"]]

In [ ]:
unique_oclcs = []
unique_oclcs.extend(first_50_oclc)

In [ ]:
second_20 = session.search_brief_bibs(q=query_test, inCatalogLanguage=None, offset=51, limit=24, orderBy="bestMatch")

In [ ]:
len(first_50.json()["briefRecords"]), len(second_20.json()["briefRecords"])

In [ ]:
second_20_oclc = [x["oclcNumber"] for x in second_20.json()["briefRecords"]]

In [ ]:
[x for x in second_20_oclc if x in first_50_oclc]

In [ ]:
unique_oclcs.extend(second_20_oclc)

In [ ]:
# found all the unique ones in one order by setting limit=24

# for x in range(1,25):
#     res = session.search_brief_bibs(q=query_test, inCatalogLanguage=None, offset=50, limit=x)
#     oclc = [y["oclcNumber"] for y in res.json()["briefRecords"]]
#     [unique_oclcs.append(z) for z in oclc if z not in unique_oclcs]

In [ ]:
offset_lim_df = pd.DataFrame(index=unique_oclcs)
offset_lim_df.insert(loc=0, column="idx", value=list(range(1, len(unique_oclcs) + 1)))

In [ ]:
for x in range(1,25):
    res = session.search_brief_bibs(q=query_test, inCatalogLanguage=None, offset=51, limit=x, orderBy="bestMatch")
    oclc = [y["oclcNumber"] for y in res.json()["briefRecords"]]
    oclc_series = pd.Series(index=oclc, data=[x for x in range(1, len(oclc) + 1)], name=str(x))
    offset_lim_df = offset_lim_df.join(oclc_series)

In [ ]:
pd.options.display.max_columns = 25
pd.options.display.max_rows = 75

In [ ]:
# mwh_offset_lim_df = offset_lim_df.copy()

In [ ]:
offset_test = session.search_brief_bibs(q=query_test, inCatalogLanguage=None, offset=51, limit=20, orderBy="bestMatch")
len(offset_test.json()["briefRecords"])

In [ ]:
count_df = pd.DataFrame(offset_lim_df.count()).T.rename(index={0:"Count"})
exp_count_df = count_df.rename(index={"Count": "Expected Count"})
exp_count_df.loc["Expected Count"] = np.arange(25)
exp_count_df.loc["Expected Count", "idx"] = 70

In [ ]:
display_df = pd.concat([offset_lim_df, count_df, exp_count_df]).rename(columns={"idx":"Record#"}).rename_axis(index="oclcNumber", columns="&limit=")
display_df

In [ ]:
idx = pd.IndexSlice
slice_ = idx[idx["760523067":"300845905"], idx["1":"24"]]
display_df.style.hide(subset=slice("680615393", "300642591"), axis=0) \
            .format(precision=0) \
            .background_gradient(axis=None, vmin=0, vmax=40, cmap="BuGn", subset=slice_).highlight_null("white")

### Result selection

Once the results are back from Worldcat we need to pick which to use if multiple have been returned. For Urdu this is less of an issue as there are fewer results returned. For Chinese there are in some instances upwards of 500 records so we do need to select.

Victoria Morris suggested:
- Leader position 17
    - Encoding level, avoid 3/5/7
- 016 where \$2 subfield contains Uk (existing BL holding)
    - 16 is National bibliographic agency control number, $2Uk indicates it's the BL
- 040 
    - Cataloguing source (if "OCLC" appears lots of times then that's good)
- 042
    - Authentication code (quality control process)
- 100/110 and 245 (Searching on these)
    - Author/Corporate Author and Title
- 264/260
    - Old/new format publication information - 260 preferable to 264
- 300
    - Physical description
- 6XX 
    - Subject access
- 880 fields (linked to 100/245 or otherwise)
    - Linked to other fields, indicating original text formats

In [ ]:
def n_str(record, string):
    if type(record) is Record:
        f040 = record.get_fields("040")[0].text().split(" ")
        oclc_occurences = sum([1 for x in f040 if string in x])
        return oclc_occurences
    else:
        return None

    
def len_record(record):
    return len(record.get_fields())


def apply_n_str(x, string):
    if type(x) is dict:
        return [n_str(v, string) for v in x.values()]
    else:
        return None

    
def apply_len_record(x):
    if type(x) is dict:
        return [len_record(v) for v in x.values()]
    else:
        return None
    
    
def blx_holdings_exist(record):
    no_holdings = "NO HOLDINGS IN BLX" in record.get_fields("948")[0].text()
    if record.get_fields("016"):
        bl_system_number = "$2Uk" not in record.get_fields("016")[0].__str__()
    if no_holdings or bl_system_number:
        return 0
    else:
        return 1

In [ ]:
len(all_records)

In [ ]:
blx_holdings = [blx_holdings_exist(r) for r in all_records]

In [ ]:
sum(blx_holdings)

In [ ]:
oclc_count = res.apply(apply_n_str, string="OCLC")
bl_count = res.apply(apply_n_str, string="BLX")
record_len = res.apply(apply_len_record)

In [ ]:
cards_df["040_oclc_count"] = oclc_count
cards_df["040_bl_count"] = bl_count
cards_df["record_len"] = record_len

In [ ]:
sum(cards_df['040_bl_count'].dropna().sum())

In [ ]:
record_len.dropna().apply(lambda x: len(x)).describe()

In [ ]:
record_len.dropna().apply(lambda x: len(x)).hist(bins=40)

In [ ]:
cards_df.dropna(subset=["040_oclc_count"])

In [ ]:
oclc_count_df = pd.DataFrame(
    data={
        "040_oclc_count": cards_df['040_oclc_count'].dropna().sum(),
        "record_len": cards_df['record_len'].dropna().sum(),
    }
)

In [ ]:
oclc_count_df.shape

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter("record_len", "040_oclc_count", data=oclc_count_df)
ax.set_xlabel("Fields in record", fontsize='x-large')
ax.set_ylabel("Occurences of 'OCLC' in MARC field 040", fontsize='x-large')
ax.set_title("Relationship between number of occurences of 'OCLC'\nin MARC field 040 in a record and total fields in a record", fontsize="x-large")
ax.tick_params(labelsize="x-large")

In [ ]:
# fig.savefig("..//reports//figures//OCLC_record_length.png", dpi=300, bbox_inches="tight")